In [1]:
# Importing the libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization

In [2]:
(train_data, test_data), info = tfds.load('imdb_reviews', split=['train', 'test'], with_info=True, as_supervised=True)

In [3]:
type(test_data)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

# Shuffling the data for training and creating batches of text and label pairs

In [4]:
def prepare_dataset(data, batch_size=32, buffer_size=10000):
    return data.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
train_dataset = prepare_dataset(train_data)
test_dataset = prepare_dataset(test_data)

In [6]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [7]:
vocab_size = 10000
sequence_length = 100

# Encoding the text data

In [8]:
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [9]:
train_text = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_text)

# Building a sequential model using tf.keras.Sequential function

In [10]:
model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, 64, mask_zero=True),
    SimpleRNN(64),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compiling the model

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Training the model

In [12]:
epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 53ms/step - accuracy: 0.5061 - loss: 0.6902 - val_accuracy: 0.5000 - val_loss: 0.6828
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.5396 - loss: 0.6673 - val_accuracy: 0.5182 - val_loss: 0.7003
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.6142 - loss: 0.6256 - val_accuracy: 0.5825 - val_loss: 0.6685
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.6998 - loss: 0.5558 - val_accuracy: 0.6193 - val_loss: 0.6732
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.7679 - loss: 0.4696 - val_accuracy: 0.6254 - val_loss: 0.7605
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.8379 - loss: 0.3584 - val_accuracy: 0.6209 - val_loss: 0.7638
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.8752 - loss: 0.2804 - val_accuracy: 0.5850 - val_loss: 0.8791
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.8785 - loss: 0.2773 - 

# Evaluating the model on the test dataset

In [13]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5899 - loss: 1.2021
Test Loss: 1.2159901857376099
Test Accuracy: 0.5851600170135498


In [19]:
def predict_sentiment(sentence):
    prediction = model.predict(tf.convert_to_tensor([sentence]))[0][0]
    if prediction >= 0.0:
        return "Positive"
    else:
        return "Negative"

# Testing the model by passing a sentence

In [21]:
example_sentence = input("Enter the test sentence: ")
print(predict_sentiment(example_sentence))

Enter the test sentence: Good Movie!!!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Positive
